In [10]:
import gym
import time
import numpy as np

In [2]:
env = gym.make('CartPole-v1')
env.reset()

for step in range(100):
    env.render()
    action = env.action_space.sample()
    # sample: (array([ 0.20307615,  0.24289648, -5.11968664, -5.43760231]), 0.0, True, {})
    observation, reward, done, info = env.step(action)
    time.sleep(0.02)
env.close()

/home/i-sip_iot/.local/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
env.step(env.action_space.sample())

(array([ 0.23074171, -0.10530487, -6.34700909, -1.97411981]), 0.0, True, {})

In [4]:
env.action_space.n

2

In [9]:
env.observation_space

Box(4,)

In [ ]:
action_size = env.action_space.n 

In [5]:
def craete_bins(number_of_bins=10):
    bins_cart_position = np.

AttributeError: 'Box' object has no attribute 'n'